# Simulate parabolic PDE with DEM as parameter field

Idea: use sandbox DEM as paramerter field in a parabolic PDE simulation

### import the libraries

In [ ]:
import os,sys
import matplotlib.pyplot as plt
sys.path.append('./../../../open_AR_Sandbox')
import sandbox.sandbox as sb

### Setup the projector and Sensor and load a calibration

In [ ]:
calib = sb.CalibrationData(file = "lunch_lehre_calibration.json")
sensor = sb.KinectV2(calib)
projector = sb.Projector(calib)

In [ ]:
module = sb.TopoModule(calib,sensor,projector)
module.setup()

### start prototyping 

In [ ]:
import numpy as np
from scipy import sparse  # to define sparse matrices
import scipy.sparse.linalg as ssl
from scipy.sparse import (csr_matrix, coo_matrix, dia_matrix, lil_matrix,
                              dok_matrix, rand, SparseEfficiencyWarning)

In [ ]:
def poisson2d_param(N, M, params, dtype='d', format='csc'):
    """
    Return a sparse matrix for the 2D Poisson problem
    with standard 5-point finite difference stencil on a
    square N-by-N grid.
    
    Consider heterogeneities in parameter field (argument params)
    """
    if N == 1:
        diags = np.asarray([[4]], dtype=dtype)
        return dia_matrix((diags, [0]), shape=(1, 1)).asformat(format)

    offsets = np.array([0, -N, N, -1, 1])

    diags = np.empty((5, N*M), dtype=dtype)

    diags[0] = -4  # main diagonal (note: in example it is 4)
    diags[1:] = 1  # all offdiagonals (note: in example, it is -1!)
        
    # temper with parameter fields:
    diags[0] = - 4 * np.ravel(params)
    
    # param_frame = np.ones_like(params) * 4.
    # param_frame[1:-1, 1:-1] = params[:-2, 1:-1] + params[2:, 1:-1] + params[1:-1, 2:] + params[1:-1, :-2]
    
    param_comb = params[:-2, 1:-1] + params[2:, 1:-1] + params[1:-1, 2:] + params[1:-1, :-2]
    param_frame = np.pad(param_comb, (1,1), 'edge')
    
    order_flag = 'F'   # {'C','F', 'A', 'K'}
    
    diags[0] = - np.ravel(param_frame, order=order_flag)
    
    # print(diags[0])
    
    diags[1] = np.ravel(params, order=order_flag)
    diags[2] = np.ravel(params, order=order_flag)
    diags[3] = np.ravel(params, order=order_flag)
    diags[4] = np.ravel(params, order=order_flag)
    
    # set Neumann BCs
    # diags[1, N-1::N] = 2
    diags[3, N-2::N] = 2 * diags[3, N-2::N]
    # diags[1, (N-1)**2:(N-1)**2+N] = 2
    # diags[2, :2*N] = 2
    diags[4, 1::N] = 2 * diags[4, 1::N]
    # diags[3, (N-1)*N:] =2
    
    # diags[3, :] = 2
    
    # as before:
    diags[3, N-1::N] = 0  # first lower diagonal
    diags[4, N::N] = 0  # first upper diagonal
    
    # print(diags)

    return dia_matrix((diags, offsets), shape=(N*M, N*M)).asformat(format)



def scale_dem(sandbox_dem):
    params = 1 + (sandbox_dem - np.min(sandbox_dem)) /\
    (np.max(sandbox_dem) - np.min(sandbox_dem))
    return params**2

def run_sim(params):
    n = 150
    m = 200
    n, m = params.shape
    T = poisson2d_param(n, m, params)
    # set value with boundary conditions
    b = np.zeros(n * m)
    b_val = 20
    b[:n] = -b_val
    b[-n:] = b_val
    # solve
    u = ssl.spsolve(T, b)
    u2 = u.reshape(m,n)
    return u2

def plot_field(u):
    n,m = u.shape
    x = np.linspace(0, 10, n)
    y = np.linspace(0, 10, m)
    X,Y = np.meshgrid(x,y)
    # fig = plt.figure(figsize=(12,12))
    # ax = fig.add_subplot(111, projection='3d')
    # ax.plot_surface(X, Y, u2, cmap='viridis', cstride=2, rstride=2) # my new favourite colormap

    fig = plt.figure(figsize=(14,8))
    # 
    # plt.contourf(sandbox_dems[dem_id,:,:])
    plt.contourf(u.T, 25, cmap='RdBu')
    plt.axis('equal')
    plt.colorbar()
    plt.xlim([0,200])
    plt.ylim([0,150])
    plt.tight_layout()

def plot_quiver(u):
    n,m = u.shape
    n_steps = 10
    y = np.linspace(0,m,int(m/n_steps))
    x = np.linspace(0,n,int(n/n_steps))
    X,Y = np.meshgrid(x,y)

    fig = plt.figure(figsize=(12,8))
    plt.contourf(params, 25)
    plt.quiver(X, Y, -np.gradient(u)[0][::n_steps,::n_steps],  -np.gradient(u)[1][::n_steps,::n_steps])
    plt.axis('equal')
    plt.xlim([0,200])
    plt.ylim([0,150])
    plt.tight_layout()

def plot_streamlines(u,plot_lines=True, savefig=False, fig_filename="dtm_streamlines.png"):
    n,m = u.shape
    n_steps = 10
    y = np.linspace(0,m,int(m/n_steps+1))
    x = np.linspace(0,n,int(n/n_steps+1))
    X,Y = np.meshgrid(x,y)
    
    # print(X.shape)

    fig = plt.figure(figsize=(12,8))
    # plt.contourf(u2.T)
    plt.contourf(params, 25)
    plt.colorbar()
    # plt.quiver(X, Y, -np.gradient(u2)[0].T[::n_steps,::n_steps],  -np.gradient(u2)[1].T[::n_steps,::n_steps])
    plt.axis('equal')
    speed = 1/np.sqrt(np.gradient(u)[0].T[::n_steps,::n_steps]**2 + np.gradient(u)[1].T[::n_steps,::n_steps]**2)
    lw = 10*speed / speed.max()
    tmp = -np.gradient(u)[0].T[::n_steps,::n_steps]
    # print(tmp.shape)
    if plot_lines:
        plt.streamplot(X, Y,-np.gradient(u)[0].T[::n_steps,::n_steps],  -np.gradient(u)[1].T[::n_steps,::n_steps],
                      linewidth=lw,
                      color='w',
                       arrowsize = 2.,
                      integration_direction='forward')
    # plt.xlim([0,200])
    # plt.ylim([0,150])
    plt.tight_layout()
    if savefig:
        plt.savefig(fig_filename)

In [ ]:
depth = sensor.get_frame()
cropped_depth = module.clip_frame(module.crop_frame(depth))

In [ ]:
plt.pcolormesh(cropped_depth)

In [ ]:
params = scale_dem(cropped_depth)
u = run_sim(params)

In [ ]:
# plot_field(u)

plot_streamlines(u)
